# Calculating Returns for an Index

This notebook shows how LUSID can calculate returns for a Reference Portfolio

In [ ]:
# Import common libraries
import os
import pandas as pd
import logging
import pytz
import json
import random
from datetime import datetime, timezone
from IPython.core.display import HTML
logging.basicConfig(level = logging.INFO)

# Import LUSID libraries
import lusid as lu
import lusid.models as lm

import lusidjam
import lusid.extensions as le
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils.lpt.lpt import to_date
from finbourne_sdk_utils import cocoon as cocoon

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    le.ArgsConfigurationLoader(access_token = lusidjam.RefreshingToken(), app_name = "LusidJupyterNotebook"),
    le.EnvironmentVariablesConfigurationLoader(),
    le.SecretsFileConfigurationLoader(secrets_path)]
api_factory = le.SyncApiClientFactory(config_loaders=config_loaders)

# Confirm success
api_client = api_factory.build(lu.ApplicationMetadataApi)
api_url = api_client.api_client.configuration._base_path.replace("api","")

print ('LUSID Environment :', api_url + "docs")
display(pd.DataFrame(api_client.get_lusid_versions().to_dict()))

In [ ]:
portfolios_api = api_factory.build(lu.PortfoliosApi)
reference_portfolios_api = api_factory.build(lu.ReferencePortfolioApi)
instruments_api = api_factory.build(lu.InstrumentsApi)
quotes_api = api_factory.build(lu.QuotesApi) 
configuration_recipe_api = api_factory.build(lu.ConfigurationRecipeApi)

In [ ]:
scope = "index-returns"
code = "demo-index"
recipe_code = "index-returns-recipe"
start_q1 = to_date("2020-01-01")
start_q2 = to_date("2020-01-03")
quote_type="Price"
quote_field="mid"

## 1. Data load

### 1.1 Load CSV files of transaction and constituent data

In [ ]:
# Load CSV file of constituents
constituent_file_csv = r"data/index-returns/constituents.csv"
constituent_df = pd.read_csv(constituent_file_csv)
constituent_df

### 1.2 Create reference portfolio

In [ ]:
try:
    reference_portfolios_api.create_reference_portfolio(
            scope=scope,
            create_reference_portfolio_request=lm.CreateReferencePortfolioRequest(
                display_name="Demo Index", 
                code=code, 
                created=start_q1.isoformat(), 
                instrument_scopes=[scope]
            ),
        )
    
    print("Portfolios created")
    
except lu.ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] == 112:  # PortfolioWithIdAlreadyExists
            print(f"Portfolio {scope}/{code} already exists")
        else:
            raise e

### 1.3 Create instrument master

In [ ]:
instrument_id_mapping = {
    'instrument_id': 'ClientInternal',
    'ticker': 'Ticker',
    'sedol': 'Sedol',
}

request={}

# Read the listed instruments from the Excel sheet
for _, row in constituent_df.iterrows():
    request[row['instrument_id']] = lm.InstrumentDefinition(
        name=row['instrument_name'],
        identifiers={
            instrument_id: lm.InstrumentIdValue(value=row[source])
            for source, instrument_id in instrument_id_mapping.items()
            if not pd.isna(row[source])
        },
        definition=lm.SimpleInstrument(
            instrument_type="SimpleInstrument",
            dom_ccy="GBP",
            asset_class="Equities",
            simple_instrument_type="Listed Equity"
        ),
        properties=[]
    )

response = instruments_api.upsert_instruments(
    request_body=request,
    scope=scope)

print("Instruments Upserted")

### 1.4 Upload constituents

In [ ]:
# Initialise a list to hold our constituents
constituents = []

for index, row in constituent_df.iterrows():
    constituents.append(
        lm.ReferencePortfolioConstituentRequest(
            instrument_identifiers={
                "Instrument/default/ClientInternal": row["instrument_id"]
            },
            weight=row["weight"],
            currency="GBP",
        )
    )

constituents_request = lm.UpsertReferencePortfolioConstituentsRequest(
    effective_from=start_q1.isoformat(),
    weight_type="Static",
    constituents=constituents,
)

response = reference_portfolios_api.upsert_reference_portfolio_constituents(
    scope=scope,
    code=code,
    upsert_reference_portfolio_constituents_request=constituents_request,
)

print("Constituents Upserted")

### 1.5 Upload quotes

In [ ]:
quotes_for_upsert = {}

for index, row in constituent_df.iterrows():
    quotes_for_upsert[f"{row['instrument_id']}_start"] = lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row['instrument_id'],
                instrument_id_type="ClientInternal",
                quote_type=quote_type,
                field=quote_field,
            ),
            effective_at=start_q1.isoformat(),
        ),
        metric_value=lm.MetricValue(value=row['start_price'], unit="GBP"),
    )
    quotes_for_upsert[f"{row['instrument_id']}_end"] = lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row['instrument_id'],
                instrument_id_type="ClientInternal",
                quote_type=quote_type,
                field=quote_field,
            ),
            effective_at=start_q2.isoformat(),
        ),
        metric_value=lm.MetricValue(value=row['end_price'], unit="GBP"),
        )
    
upsert_quotes_response = quotes_api.upsert_quotes(
    scope=scope, request_body=quotes_for_upsert
)

print("Quotes Upserted")

### 1.6 Create a Recipe

In [ ]:
recipe = lm.ConfigurationRecipe(
    scope=scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type=quote_type,
                field=quote_field,
                quote_interval="100D.0D"
            )
        ]
    )
)

upsert_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(configuration_recipe=recipe)
)

print("Recipe upserted")

## 2. Access Data

### 2.2 LUSID to calculate the Portfolio-level return for the Reference Portfolio based on the price changes

Please note that the existing API is not performance tested at large volumes.  Anecdotal evidence suggests the performance is acceptable for a Reference Portfolio containing up to 50 constituents over a period of 5 years.

In [ ]:
returns=portfolios_api.get_portfolio_aggregated_returns(
    scope=scope,
    code=code,
    from_effective_at=start_q1.isoformat(),
    to_effective_at=start_q2.isoformat(),
    aggregated_returns_request=lm.AggregatedReturnsRequest(
        recipe_id=lm.ResourceId(
            scope=scope,
            code=recipe_code
        ),
        metrics=[
            lm.PerformanceReturnsMetric(
                type="Return",
                window="1D",
                alias="1D")
        ]
    )
)

output = pd.DataFrame(returns.results['index-returns/demo-index'])
output


### 2.1 View Reference Portfolio in website 

In [ ]:
display(HTML(f'<a href="{api_url}app/dashboard/holdings?scope={scope}&code={code}&entityType=Portfolio" target="_blank">See Reference Portfolio Constituents</a>'))

display(HTML(f'<a href="{api_url}app/dashboard/returns?dateFrom={start_q1.date().isoformat()}&dateTo={start_q2.date().isoformat()}&frequency=Daily&compositeMethod=Asset&period=Daily&scope={scope}&code={code}&entityType=Portfolio&returnsScope=returns&returnsCode=daily_return&returnsMetrics=1D,Inception,Amt,MTD,1M,1M-partial&recipeScope={scope}&recipeCode={recipe_code}" target="_blank">Open Returns Dashboard</a>'))